In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.prod import ProdDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn import MMPFNClassifier
from mmpfn.models.mmpfn.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/anaconda3/envs/mmpfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/prod")
dataset = ProdDataset(data_path)
_ = dataset.get_embeddings()

Load embeddings from embeddings/prod/prod.pt


In [3]:
accuracy_scores = []
for seed in range(10):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices].unsqueeze(1)
    image_test = test_dataset.dataset.embeddings[test_dataset.indices].unsqueeze(1)
    
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_pad_ufes_20.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=None,
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM' # no_append token_append split_append multihead cross_merger glu
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM' # no_append token_append split_append multihead cross_merger glu
    )

    clf_finetuned = model_finetuned.fit(None, image_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(None, image_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:  52%|█████▏    | 52/100 [04:13<03:58,  4.97s/it, Best Val. Loss=0.925, Best Val. Score=-0.925, Training Loss=0.925, Val. Loss=0.925, Patience=-1, Utilization=0, Grad Norm=0.826] [2025-09-09 17:22:05,143] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  56%|█████▌    | 56/100 [04:32<03:34,  4.87s/it, Best Val. Loss=0.925, Best Val. Score=-0.925, Training Loss=0.919, Val. Loss=0.925, Patience=-4, Utilization=0, Grad Norm=1.71] [2025-09-09 17:22:23,938] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  84%|████████▍ | 84/100 [06:52<01:21,  5.11s/it, Best Val. Loss=0.924, Best Val. Score=-0.924, Training Loss=0.905, Val. Loss=0.924, Patience=-31, Utilization=0, Grad Norm=2.19] [2025-09-09 17:24:43,687] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [08:18,  4.99s/it, Best Val. Loss=0.924, Best Val. Score=-0.924, Training Loss=0.922, Val. Loss=0.924, Patience=

accuracy_score (Finetuned): 0.5993715632364494


Fine-tuning Steps: 101it [08:16,  4.97s/it, Best Val. Loss=0.923, Best Val. Score=-0.923, Training Loss=0.926, Val. Loss=0.926, Patience=-50, Utilization=0, Grad Norm=0.116]                         
[2025-09-09 17:34:31,253] INFO - Initial Validation Loss: 0.9243203160555957 Best Validation Loss: 0.9233161188086447 Total Steps: 101 Best Step: 2 Total Time Spent: 498.4231262207031


accuracy_score (Finetuned): 0.5946582875098193


Fine-tuning Steps:   5%|▌         | 5/100 [00:25<09:53,  6.25s/it, Best Val. Loss=0.918, Best Val. Score=-0.918, Training Loss=0.918, Val. Loss=0.925, Patience=46, Utilization=0, Grad Norm=0.228]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.6369565217391304
Std Accuracy: 0.02184755569808889
